---
title: IDs from STEREO
---

## Setup

Need to run command in shell first as `pipeline` is project-specific command

```{sh}
kedro pipeline create stearo
```

To get STEREO-A state data, run `kedro run --to-outputs=sta.primary_state_rtn_1h`

To get candidates data, run `kedro run --from-inputs=sta.feature_1s --to-outputs=candidates.sta_1s`

In [ ]:
#| default_exp pipelines/stearo/pipeline

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| code-summary: import all the packages needed for the project
#| export
#| output: hide

from fastcore.utils import *
from fastcore.test import *
from ids_finder.utils.basic import *
from ids_finder.core import *

import polars as pl
import pandas
import numpy as np

from datetime import timedelta
from loguru import logger

from typing import Callable, Dict

[10/24/23 14:51:05] WARNING  TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook    ]8;id=317044;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=51268;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter                  
                             console)                                                                              
                                                                                                                   

In [ ]:
#| export
from ids_finder.core import extract_features

#### `Kerdo`

In [ ]:
#| export
from kedro.pipeline import Pipeline, node
from kedro.pipeline.modular_pipeline import pipeline

In [ ]:
#| eval: false
from ids_finder.utils.basic import load_catalog

catalog = load_catalog()

[10/24/23 14:51:06] WARNING  KedroDeprecationWarning: 'PartitionedDataset' has been moved to        ]8;id=273524;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=241962;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             `kedro-datasets` and will be removed in Kedro 0.19.0.                                 
                                                                                                                   

                    WARNING  KedroDeprecationWarning: 'PartitionedDataset' has been moved to        ]8;id=762659;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=335193;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             `kedro-datasets` and will be removed in Kedro 0.19.0.                                 
                                                                                                                   

                    WARNING  KedroDeprecationWarning: 'AbstractVersionedDataSet' has been renamed   ]8;id=194900;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=634475;file:///Users/zijin/miniforge3/envs/cool_planet/lib/python3.10/warnings.py#109\109]8;;\
                             to 'AbstractVersionedDataset', and the alias will be removed in Kedro                 
                             0.19.0                                                                                
                                                                                                                   

## Pipelines

In [ ]:
#| export
from ids_finder.pipelines.stereo.mag import create_pipeline as create_mag_data_pipeline
from ids_finder.pipelines.stereo.state import create_pipeline as create_state_data_pipeline
from ids_finder.pipelines.default.mission import create_combined_data_pipeline

In [ ]:
# | export
def create_pipeline(
    sat_id="sta",
    tau="60s",
    ts_mag="1s",  # time resolution of magnetic field data
    ts_state="1h",  # time resolution of state data
) -> Pipeline:
    return (
        create_mag_data_pipeline(sat_id, ts=ts_mag, tau=tau)
        + create_state_data_pipeline(sat_id, ts=ts_state)
        + create_combined_data_pipeline(sat_id, tau=tau, ts_state=ts_state)
    )

## Obsolete codes

NOTE: one can also use `speasy` to download data, however this is slower for `STEREO` data.

In [ ]:
%%markdown
sat_fgm_product = cda_tree.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD
sat_fgm_product = 'cda/STA_L1_MAG_RTN/BFIELD'
products = [sat_fgm_product]

dataset = spz.get_data(products, test_trange, disable_proxy=True)
sat_fgm_data  = dataset[0]
data_preview(sat_fgm_data)

Download data in a background thread

In [ ]:
%%markdown

@threaded
def download_data(products, trange):
    logger.info("Downloading data")
    spz.get_data(products, trange, disable_proxy=True)
    logger.info("Data downloaded")
    
download_data(products, trange)

In [ ]:
import speasy as spz

In [ ]:
cda_tree: spz.SpeasyIndex = spz.inventories.tree.cda
product = cda_tree.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN

logger.info(product.description)
logger.info(product.ID)
logger.info(product.BFIELD.CATDESC)
logger.info(product.BFIELD.spz_uid())

# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.
# spz.inventories.data_tree.cda.STEREO.STEREOA.IMPACT_MAG.STA_LB_MAG_RTN.description
# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.MAGFLAGUC.CATDESC
spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD.CATDESC
# spz.inventories.data_tree.cda.STEREO.Ahead.IMPACT_MAG.STA_L1_MAG_RTN.BFIELD.